<a href="https://colab.research.google.com/github/8percent/programming-bitcoin/blob/main/chapter7/%EC%98%A4%EC%9E%AC%EA%B7%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/jimmysong/programmingbitcoin

Cloning into 'programmingbitcoin'...
remote: Enumerating objects: 6406, done.
remote: Total 6406 (delta 0), reused 0 (delta 0), pack-reused 6406
Receiving objects: 100% (6406/6406), 39.43 MiB | 13.58 MiB/s, done.
Resolving deltas: 100% (4254/4254), done.


In [2]:
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
cd programmingbitcoin/code-ch07

/content/programmingbitcoin/code-ch07


In [4]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc
import helper
import tx
import script

In [5]:
from tx import Tx
from io import BytesIO
raw_tx = ('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed01210349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278afeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600')
stream = BytesIO(bytes.fromhex(raw_tx))
transaction = Tx.parse(stream)
print(transaction.fee() >= 0)

True


In [6]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
print(point.verify(z, signature))

True


In [9]:
from helper import hash256
modified_tx = bytes.fromhex('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000001976a914a802fc56c704ce87c42d7c92eb75e7896bdc41ae88acfeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac1943060001000000')
h256 = hash256(modified_tx)
z = int.from_bytes(h256, 'big')
print(hex(z))

0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6


In [8]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
point.verify(z, signature)

True

### Exercise 1

Write the `sig_hash` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sig_hash`

In [12]:
# Exercise 1

reload(tx)
run(tx.TxTest("test_sig_hash"))

.
----------------------------------------------------------------------
Ran 1 test in 0.007s

OK


### Exercise 2

Write the `verify_input` method for the `Tx` class. You will want to use the `TxIn.script_pubkey()`, `Tx.sig_hash()` and `Script.evaluate()` methods.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_verify_p2pkh`

In [13]:
# Exercise 2

reload(tx)
run(tx.TxTest("test_verify_p2pkh"))

.
----------------------------------------------------------------------
Ran 1 test in 0.351s

OK


In [14]:
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx
prev_tx = bytes.fromhex('0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299')
prev_index = 13
tx_in = TxIn(prev_tx, prev_index)
tx_outs = []
change_amount = int(0.33*100000000)
change_h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
change_script = p2pkh_script(change_h160)
change_output = TxOut(amount=change_amount, script_pubkey=change_script)
target_amount = int(0.1*100000000)
target_h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)
tx_obj = Tx(1, [tx_in], [change_output, target_output], 0, True)
print(tx_obj)

tx: cd30a8da777d28ef0e61efe68a9f7c559c1d3e5bcd7b265c850ccb4068598d11
version: 1
tx_ins:
0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299:13
tx_outs:
33000000:OP_DUP OP_HASH160 d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f OP_EQUALVERIFY OP_CHECKSIG
10000000:OP_DUP OP_HASH160 507b27411ccf7f16f10297de6cef3f291623eddf OP_EQUALVERIFY OP_CHECKSIG
locktime: 0


In [19]:
from ecc import PrivateKey
from helper import SIGHASH_ALL
transaction = tx_obj
z = transaction.sig_hash(0)
private_key = PrivateKey(secret=8675309)
der = private_key.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
sec = private_key.point.sec()
script_sig = Script([sig, sec])
transaction.tx_ins[0].script_sig = script_sig 
print(transaction.serialize().hex())

010000000199a24308080ab26e6fb65c4eccfadf76749bb5bfa8cb08f291320b3c21e56f0d0d0000006b4830450221008ed46aa2cf12d6d81065bfabe903670165b538f65ee9a3385e6327d80c66d3b502203124f804410527497329ec4715e18558082d489b218677bd029e7fa306a72236012103935581e52c354cd2f484fe8ed83af7a3097005b2f9c60bff71d35bd795f54b67ffffffff02408af701000000001976a914d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f88ac80969800000000001976a914507b27411ccf7f16f10297de6cef3f291623eddf88ac00000000


In [18]:
from ecc import PrivateKey
from helper import hash256, little_endian_to_int
secret = little_endian_to_int(hash256(b'Jimmy Song secret'))
private_key = PrivateKey(secret)
print(private_key.point.address(testnet=True))

mn81594PzKZa9K3Jyy1ushpuEzrnTnxhVg


### Exercise 3

Write the `sign_input` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sign_input`

In [23]:
# Exercise 3

reload(tx)
run(tx.TxTest("test_sign_input"))

.
----------------------------------------------------------------------
Ran 1 test in 0.258s

OK


In [25]:

phasekey = b'jaegyoon.oh@8percent.kr jkjacb12896782349013678'
phasenum = little_endian_to_int(hash256(phasekey))
print(phasenum)
pkey = PrivateKey(phasenum)
ppoint = pkey.point
print(ppoint.address(testnet = True))


33737802847435399979011524323257422982427757150152127280827273667914986388143
moF19FZzo7ewy53yeX1o6SLYnzNKCV5mcA


### Exercise 4

Create a testnet transaction that sends 60% of a single UTXO to `mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv`. The remaining amount minus fees should go back to your own change address. This should be a 1 input, 2 output transaction.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [34]:
# Exercise 4

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# create 1 TxIn and 2 TxOuts
# 1 of the TxOuts should be back to your address
# the other TxOut should be to this address
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'

# get the private key from the exercise in Chapter 4
# change address should be the address generated from Chapter 4
source_address = "moF19FZzo7ewy53yeX1o6SLYnzNKCV5mcA"
# get the prev_tx and prev_index from the transaction where you got
# some testnet coins
# create a transaction input for the previous transaction with
# the default ScriptSig and sequence
prev_tx = bytes.fromhex('22c82279b87da4610da0339db6a8f71c31b366888902dcdbab29460c12d9134f')
prev_index = 1
tx_in = TxIn(prev_tx, prev_index)
tx_outs = []
fee = 0.00001
remain = 0.001 * 0.4 - fee
change_amount = int(remain * 100000000)
change_h160 = decode_base58(source_address)
change_script = p2pkh_script(change_h160)
change_output = TxOut(amount = change_amount, script_pubkey = change_script)

target_amount = int(0.001 * 0.6 * 100000000)
target_h160 = decode_base58(target_address)
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount = target_amount, script_pubkey = target_script)
tx_obj = Tx(1, [tx_in], [change_output, target_output], 0, True)
print(tx_obj)
# target amount should be 60% of the output amount
# set the fee to some reasonable amount
# change amount = amount from the prev tx - target amount - fee

# create a transaction output for the target amount and address
# create a transaction output for the change amount and address
# create the transaction object

# sign the one input in the transaction object using the private key
# print the transaction's serialization in hex
# broadcast at http://testnet.blockchain.info/pushtx
secret = 33737802847435399979011524323257422982427757150152127280827273667914986388143
vi = tx_obj.sign_input(0, PrivateKey(secret = secret))
print(vi)
print(tx_obj.serialize().hex())

tx: 19cea754224d05edd8d57ccf715dc09050653b9cba73b3203715f972925d5abe
version: 1
tx_ins:
22c82279b87da4610da0339db6a8f71c31b366888902dcdbab29460c12d9134f:1
tx_outs:
39000:OP_DUP OP_HASH160 54bcba7d3db1faf5438473c5659d00f40ef8d6e7 OP_EQUALVERIFY OP_CHECKSIG
59999:OP_DUP OP_HASH160 ad346f8eb57dee9a37981716e498120ae80e44f7 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0
True
01000000014f13d9120c4629abdbdc02898866b3311cf7a8b69d33a00d61a47db87922c822010000006a47304402204166f6ed500b2e936cd123fdd803f9e2fb61c476818bc7557575fe8a4ee800b302207103ab784a3307d4f46d695a8e11d3ae38c26c7aba9cfbb944e03b171b66c7f0012102d7a07f6793cc9f5fc14c9b5b8e5d12e649decc25c629a087c50a58e832ba9c9dffffffff0258980000000000001976a91454bcba7d3db1faf5438473c5659d00f40ef8d6e788ac5fea0000000000001976a914ad346f8eb57dee9a37981716e498120ae80e44f788ac00000000


In [35]:
phasekey = b'jaegyoon.oh@8percent.kr 8percent gogo'
phasenum = little_endian_to_int(hash256(phasekey))
print(phasenum)
pkey = PrivateKey(phasenum)
ppoint = pkey.point
print(ppoint.address(testnet = True))


66834303925315701622282441819608227503985828091546685348203351819900300373210
n4cAkD3SnJ4pWUvguH252wD9ntqE68a9Wi


### Exercise 5

Advanced: get some more testnet coins from a testnet faucet and create a 2 input, 1 output transaction. 1 input should be from the faucet, the other should be from the previous exercise, the output can be your own address.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [38]:
# Exercise 5

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# Create 2 TxIns, 1 from the Exercise 4 and 1 from a testnet faucet
# Creat 1 TxOut to the address above
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
source_address1 = "n4cAkD3SnJ4pWUvguH252wD9ntqE68a9Wi"
source_address2 = "moF19FZzo7ewy53yeX1o6SLYnzNKCV5mcA"
# get the private key from the exercise in Chapter 4

# get the prev_tx and prev_index from the transaction where you got
# some testnet coins
# create the first transaction input with the default ScriptSig and
# sequence
# get the prev_tx and prev_index from the transaction in Exercise 4
# create the second transaction input with the default ScriptSig and
# sequence

# set the fee to some reasonable amount
# target amount should be the sum of the inputs - fee
prev_tx1 = bytes.fromhex('fb23433a1a4de35b9875e06957814db7dd921a2826b874f51d00fd9429cc3d68')
prev_tx2 = bytes.fromhex('22c82279b87da4610da0339db6a8f71c31b366888902dcdbab29460c12d9134f')
prev_index = 1
tx_in1 = TxIn(prev_tx1, prev_index)
tx_in2 = TxIn(prev_tx1, prev_index)
tx_ins = [tx_in1, tx_in2]

fee = 0.00001

target_amount = int((0.001 + 0.001 - fee) * 100000000)
target_h160 = decode_base58(target_address)
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount = target_amount, script_pubkey = target_script)
tx_outs = [target_output]
tx_obj = Tx(1, tx_ins, tx_outs, 0, True)
print(tx_obj)
# create a transaction output for the amount and address

# sign the first input using the private key
# sign the second input using the private key 
# print the transaction's serialization in hex
# broadcast at http://testnet.blockchain.info/pushtx
secret1 = 66834303925315701622282441819608227503985828091546685348203351819900300373210
secret2 = 33737802847435399979011524323257422982427757150152127280827273667914986388143
vi1 = tx_obj.sign_input(0, PrivateKey(secret = secret1))
print(vi1)
vi2 = tx_obj.sign_input(1, PrivateKey(secret = secret2))
print(vi2)
print(tx_obj.serialize().hex())

tx: 3d0c159a889bb7550d9063795926ae5a527accb6e1b91b215175de2cd31bf76f
version: 1
tx_ins:
fb23433a1a4de35b9875e06957814db7dd921a2826b874f51d00fd9429cc3d68:1
fb23433a1a4de35b9875e06957814db7dd921a2826b874f51d00fd9429cc3d68:1
tx_outs:
199000:OP_DUP OP_HASH160 ad346f8eb57dee9a37981716e498120ae80e44f7 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0
True
True
0100000002683dcc2994fd001df574b826281a92ddb74d815769e075985be34d1a3a4323fb010000006b4830450221009d9df74c4fd1e7e1aa0de5d8ac452b44a7971339773db9a1db64d7ce6c2e7b50022002670495b0248eb388bdfd9195483afa529f81dbd618d5324288e2a288318b2c0121039f732fa24495a47ef36c105c159c3eb1f0657d97ecc452f1494389f8d183e991ffffffff683dcc2994fd001df574b826281a92ddb74d815769e075985be34d1a3a4323fb010000006b483045022100d2025b00a65697c39458b531966a343b8f3e6d0f34e54582de1e276fd09a1339022029058bb585cafa1eed13cffc58c60e6e36eb0fa329c96baa7baa5410f59dc209012102d7a07f6793cc9f5fc14c9b5b8e5d12e649decc25c629a087c50a58e832ba9c9dffffffff0158090300000000001976a914ad346f8eb57dee9a37981716e4